In [1]:
using JuMP
using Cbc
using DataFrames
using Gurobi
using CSV

In [20]:
# m = Model(solver = CbcSolver())
m = Model(solver=GurobiSolver(MIPGap=.1))


#not taken items (randomly generated)
ntaken = [7, 9, 12, 13, 15, 21, 24, 27, 28, 29, 34, 40, 43, 44, 45, 47, 48, 50, 59, 64, 67, 68, 74, 76, 78, 79, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 97, 98, 101]
data = CSV.read("supermarket_data.csv")
simple_dist = CSV.read("distmatrix_advanced.csv")

v = data[:,2]

simple_dist = simple_dist[:,2:103]
c = simple_dist
    
@variables m begin
    y1[1:102], Bin #1 if item i is taken in path 1
    y2[1:102], Bin #1 if item i is taken in path 2
end

@variables m begin
    x1[1:102,1:102], Bin #1 if direct path betweeen i and j in path 1
    x2[1:102,1:102], Bin #1 if direct path betweeen i and j in path 2
end

@variables m begin
    z1[1:102]
    z2[1:102]
end

@variables m begin
    t1[1:102,1:102]
    t2[1:102,1:102]
end

@constraint(m, y1[1] == 1)
@constraint(m, y1[102] == 1)
@constraint(m, z1[1] == 0)
@constraint(m, y2[1] == 1)
@constraint(m, y2[102] == 1)
@constraint(m, z2[1] == 0)
@constraint(m, sum(y1[i] for i in 2:101) <= 15)
@constraint(m, sum(y2[i] for i in 2:101) <= 15)
@constraint(m, sum(x1[i,1] for i in 1:102) == 0)
@constraint(m, sum(x1[102,j] for j in 1:102) == 0)
@constraint(m, sum(x2[i,1] for i in 1:102) == 0)
@constraint(m, sum(x2[102,j] for j in 1:102) == 0)

for i in ntaken
    @constraint(m, y1[i] == 0)
    @constraint(m, y2[i] == 0)
end

for i in 2:101
    @constraint(m, y1[i] + y2[i] <= 1)
end

for i in 1:101
    @constraint(m, sum(x1[i,j] for j in 2:102) == y1[i])
    @constraint(m, sum(x2[i,j] for j in 2:102) == y2[i])
end

for j in 2:102
    @constraint(m, sum(x1[i,j] for i in 1:101) == y1[j])
    @constraint(m, sum(x2[i,j] for i in 1:101) == y2[j])
end

for j in 2:102
    @constraint(m, sum(t1[i,j] for i in 1:101) == z1[j])
    @constraint(m, sum(t2[i,j] for i in 1:101) == z2[j])
end

for j in 1:101
    @constraint(m, sum(t1[j,k] for k in 2:102) == (z1[j] + sum(c[j,k]x1[j,k] for k in 2:102)))
    @constraint(m, sum(t2[j,k] for k in 2:102) == (z2[j] + sum(c[j,k]x2[j,k] for k in 2:102)))
end

for j in 1:102
    for k in 2:102
        @constraint(m, t1[j,k] >= x1[j,k])
        @constraint(m, t1[j,k] <= 180x1[j,k])
        @constraint(m, t2[j,k] >= x2[j,k])
        @constraint(m, t2[j,k] <= 180x2[j,k])
    end
end

@constraint(m, z1[102] + z2[102] <= 180)

@objective(m, Max, sum(v[i]*(y1[i] + y2[i])  for i in 1:102))


solve(m)




Academic license - for non-commercial use only
Optimize a model with 42207 rows, 42024 columns and 186126 nonzeros
Variable types: 21012 continuous, 21012 integer (21012 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [9e-01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 26889 rows and 26850 columns
Presolve time: 1.22s
Presolved: 15318 rows, 15174 columns, 67970 nonzeros
Variable types: 7442 continuous, 7732 integer (7732 binary)
Found heuristic solution: objective 9.9700000
Found heuristic solution: objective 17.9800000

Root relaxation: objective 1.946900e+02, 4685 iterations, 1.48 seconds
Total elapsed time = 5.16s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  175.87247    0  140   17.98000  175.87247   878%     -   13s
H    0     0                      84.6700000  175.87247   108% 

:Optimal

In [23]:
println("Items Taken in 1st Trip (In Order):")
cnode = 1
while cnode != 102
    for i in 1:102
        if getvalue(x1[cnode,i]) == 1
            cnode = i
            if cnode != 102
                println(data[cnode,1], "(Item #: ",  cnode - 1, ", ", "Price: ", data[cnode,2] , ")")
            end
            break
        end
    end
end

println("")

println("Trip 1 Duration: ", getvalue(z1[102]))

println("")

println("Items Taken in 2nd Trip (In Order):")
cnode = 1
while cnode != 102
    for i in 1:102
        if getvalue(x2[cnode,i]) == 1
            cnode = i
            if cnode != 102
                println(data[cnode,1], "(Item #: ",  cnode - 1, ", ", "Price: ", data[cnode,2] , ")")
            end
            break
        end
    end
end

println("")

println("Trip 2 Duration: ", getvalue(z2[102]))

println("")

println("Objective Value: ", getobjectivevalue(m))

println("Total Time: ", getvalue(z1[102]) + getvalue(z2[102]))



Items Taken in 1st Trip (In Order):
Swiffer Refills(Item #: 53, Price: 7.99)
Salmon(Item #: 95, Price: 5.99)
Glaceau SmartWater(Item #: 69, Price: 6.99)
Tomatoes (per lb)(Item #: 99, Price: 2.99)
Bottled Water (24)(Item #: 70, Price: 4.99)
Air Freshner(Item #: 55, Price: 6.99)
Toilet Paper(Item #: 41, Price: 7.99)
Hummus(Item #: 93, Price: 3.99)
Shampoo(Item #: 40, Price: 8.99)
Chicken Breast(Item #: 90, Price: 6.98)
Salami(Item #: 89, Price: 5.99)
Granola(Item #: 7, Price: 5.49)
Olive Oil(Item #: 9, Price: 6.99)
K-Cups(Item #: 2, Price: 10.99)
Seasoning(Item #: 15, Price: 3.99)

Trip 1 Duration: 100.00000000000594

Items Taken in 2nd Trip (In Order):
Broom(Item #: 51, Price: 13.99)
Gatorade (12)(Item #: 60, Price: 6.99)
Popcorn(Item #: 57, Price: 3.49)
Redbull (4)(Item #: 61, Price: 7.99)
Pepsi (12)(Item #: 65, Price: 5.99)
Milk Gallon(Item #: 71, Price: 3.79)
LaCroix (12)(Item #: 64, Price: 5.49)
Ritz(Item #: 62, Price: 3.99)
Dish Soap(Item #: 48, Price: 2.99)
Ibuprofen(Item #: 32, P